In [0]:
!pip install -q transformers

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import pandas as pd

from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelWithLMHead, AutoTokenizer
from transformers.optimization import AdamW

from PIL import Image
import requests
from io import BytesIO
from IPython.display import display
from multiprocessing.pool import ThreadPool

from collections import namedtuple
from time import perf_counter
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!cp -r drive/My\ Drive/image_captioning/* .

In [0]:
transform = transforms.Compose([
     transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

class GCCDataset(Dataset):
    def __init__(self, path='data/Train_GCC-training.tsv', tokenizer=None, max_length=512, timeout=2):
        self.data = pd.read_csv(path, sep='\t', header=None, names=['desc', 'url'])
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        desc, url = self.data.iloc[idx]
        text_tok_ids = self.tokenizer.encode(desc, max_length=self.max_length)

        return url, text_tok_ids

In [0]:
def get_dataloader(dataset, batch_size, pad_elem, img_transform, timeout=3, num_workers=4):
    def pad(seq, max_len, pad_elem):
        return seq + [pad_elem] * max(0, max_len - len(seq))

    def get_image_from_url(url):
        try:
            response = requests.get(url, timeout=timeout)
            if response.ok and response.url == url:
                img = Image.open(BytesIO(response.content))
                return img_transform(img)
            return None
        except Exception as e:
            return None
    
    def collate_fn(batch_data):
        list_urls, list_ids = list(zip(*batch_data))

        list_imgs = ThreadPool(num_workers).imap_unordered(get_image_from_url, list_urls) 
        list_imgs, list_ids = list(zip(*[(img, ids) for img, ids in zip(list_imgs, list_ids) 
                                                    if img is not None]))
        
        batch_img = torch.stack(list_imgs, 0)        

        batch_len = max(map(len, list_ids))
        batch_ids = torch.tensor(
            [pad(ids, batch_len, pad_elem) 
             for ids in list_ids]
        ).long()
        batch_mask = batch_ids.ne(pad_elem).int()

        return batch_img, batch_ids, batch_mask

    return DataLoader(
        dataset=dataset, batch_size=batch_size, shuffle=True,
        collate_fn=collate_fn, pin_memory=True, num_workers=0 
    )

In [0]:
def get_dataloader(dataset, batch_size, pad_elem, img_transform, timeout=3, num_workers=4):
    def pad(seq, max_len, pad_elem):
        return seq + [pad_elem] * max(0, max_len - len(seq))

    def get_image_from_url(url):
        try:
            response = requests.get(url, timeout=timeout)
            if response.ok and response.url == url:
                img = Image.open(BytesIO(response.content))
                return img_transform(img)
            return None
        except Exception as e:
            return None
    
    def collate_fn(batch_data):
        list_urls, list_ids = list(zip(*batch_data))

        list_imgs = ThreadPool(num_workers).imap_unordered(get_image_from_url, list_urls) 
        list_imgs, list_ids = list(zip(*[(img, ids) for img, ids in zip(list_imgs, list_ids) 
                                                    if img is not None]))
        
        batch_img = torch.stack(list_imgs, 0)        

        batch_len = max(map(len, list_ids))
        batch_ids = torch.tensor(
            [pad(ids, batch_len, pad_elem) 
             for ids in list_ids]
        ).long()
        batch_mask = batch_ids.ne(pad_elem).int()

        return batch_img, batch_ids, batch_mask

    return DataLoader(
        dataset=dataset, batch_size=batch_size, shuffle=True,
        collate_fn=collate_fn, pin_memory=True, num_workers=0 
    )

In [0]:
class Embedding(nn.Module):
    def __init__(self, cnn, embed_size):
        super().__init__()
        self.cnn = cnn
        self.embed_size = embed_size
        _, last_module = list(cnn.named_children())[-1]
        self.proj = nn.Linear(last_module.out_features, embed_size)

    def forward(self, x):
        x = self.cnn(x)
        return self.proj(x)

    def train(self, mode=True):
        self.cnn.eval()
        self.proj.train(mode)
        for param in self.cnn.parameters():
            param.requires_grad_(False)

In [0]:
SPECIAL_TOKENS = ('img', 'desc', 'pad')
SpecialIds = namedtuple('speacil_ids', SPECIAL_TOKENS)

In [0]:
class Image2TextDescriptor(nn.Module):
    def __init__(self, pretrained_model_text='gpt2', 
                     pretrained_model_image='resnet18'):
        super().__init__()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.tokenizer, self.special_ids = self._build_tokenizer(pretrained_model_text)
        self.gpt2 = self._build_gpt2(len(self.tokenizer), pretrained_model_text)

        cnn = self._build_cnn(pretrained_model_image)
        self.embedding = Embedding(cnn, self.gpt2.config.hidden_size)#.to(device)
        self.to(device)

    def save(self, path='weights/model_full_trained'):
        state = {
            'gpt_dict': self.gpt2.state_dict(),
            'projection_dict': self.embedding.proj.state_dict(),
            'tokenizer': self.tokenizer,
        }
        torch.save(state, path)
        print('!!! model is saved !!!')

    def load(self, path='weights/model_full_trained'):
        state = torch.load(path)
        self.gpt2.load_state_dict(state['gpt_dict'])
        self.embedding.proj.load_state_dict(state['projection_dict'])
        self.tokenizer = state['tokenizer']
        print('!!! model weights are loaded !!!')

    def forward(self, img_data, text_tok_ids=None, attn_mask=None):
        context_embeds = self.get_context_embeds(img_data)
        labels_ids = None
        if text_tok_ids is not None:
            text_embeds = self.gpt2.transformer.wte(text_tok_ids)
            full_embeds = torch.cat((context_embeds, text_embeds), dim=1)
            if self.training:
                labels_ids = self.get_labels(
                    full_embeds.shape[:-1],
                    context_embeds.size(1),
                    text_tok_ids
                ).to(self.device)
        else:
            full_embeds = context_embeds
        
        out = self.gpt2(inputs_embeds=full_embeds, labels=labels_ids, attention_mask=attn_mask)        
        return out

    def get_context_embeds(self, img_data):
        img_tok_emb, desc_tok_emb = self.gpt2.transformer.wte(
            torch.tensor([self.special_ids.img, self.special_ids.desc]).to(self.device)
        )

        img_embeds = self.embedding(img_data)[:, None, :] # new shape: (batch_size, seq_len, embed_size)
        context_embeds = torch.cat((img_tok_emb.expand_as(img_embeds), 
                                    img_embeds, 
                                    desc_tok_emb.expand_as(img_embeds)), dim=1)
        return context_embeds

    def get_labels(self, shape, context_len, tok_ids):
        labels_ids = torch.empty(shape, dtype=torch.long).fill_(-100) # -100 is mask value for labels in hugginface models
        mask = (tok_ids == self.special_ids.pad)
        labels_ids[:, context_len:] = tok_ids.masked_fill(mask, -100)

        return labels_ids

    def _build_tokenizer(self, pretrained_model):
        special_dct = {t: f"<{t.upper()}>" for t in SPECIAL_TOKENS}
        tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        tokenizer.add_special_tokens({'additional_special_tokens': list(special_dct.values())})
        special_ids = SpecialIds(**{k: tokenizer.encode(v)[0] for k, v in special_dct.items()})
        
        return tokenizer, special_ids

    def _build_gpt2(self, vocab_size, pretrained_model):
        gpt2 = AutoModelWithLMHead.from_pretrained(pretrained_model)
        gpt2.resize_token_embeddings(vocab_size)
        
        return gpt2

    def _build_cnn(self, pretrained_model):
        if pretrained_model == 'resnet18':
            return models.resnet18(pretrained=True)
        else:
            raise ValueError(f'{pretrained_model} is not supported yet :(')

In [0]:
from collections import Counter

In [0]:
c = Counter([str(b) for b in [1, 2, 5, 1, 9, 19, 1, 1, 4, 5, 5]])

In [0]:
list(sorted(c.items(), key=lambda x: x[-1], reverse=True))

[('1', 4), ('5', 3), ('2', 1), ('9', 1), ('19', 1), ('4', 1)]

In [0]:
def debug_memory():
    import collections, gc, resource, torch
    print('maxrss = {}'.format(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter((str(o.device), o.dtype, tuple(o.shape))
                                  for o in gc.get_objects()
                                  if torch.is_tensor(o))
    
    for line in sorted(tensors.items(), key=lambda x: (x[1], x[0][-1]), reverse=True):
        print('{}\t\t{}'.format(*line))

In [0]:
losses = []

def train_epoch(dataloader, model, optimizer, save_interval=20):
    torch.cuda.empty_cache()

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), leave=False)
    for idx_batch, batch in pbar:
        torch.cuda.empty_cache()
        #print('\t real size of the batch', len(batch[1]))
        #print()
        imgs, ids, mask = [x.to(model.device) for x in batch]
        '''print('#' * 10 + str(idx_batch) + '#' * 10)
        print('IMGS: ', imgs)
        print('IDS: ', ids)
        print('MASK: ', mask)'''

        loss, *_ = model(imgs, ids)#, attn_mask=mask)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        '''print('loss: ', loss.item())
        print()
        print()'''
        losses.append(loss.item())
        pbar.set_description(f'loss:{loss.item():.4f}')
        
        debug_memory()
        print('#'*50)
        if (idx_batch + 1) % save_interval == 0:
            model.save('weights/temp_weights')
            print(torch.cuda.memory_summary())

    model.save()


def train(model, dataloader, n_epochs=5, batch_size=16, lr=1e-3):
    model.load('weights/temp_weights')
    model.train()

    optimizer = AdamW(
        [param for param in model.parameters()
               if param.requires_grad], 
        lr=lr
    )

    for epoch in range(n_epochs):
        train_epoch(dataloader, model, optimizer)
        print(f'Epoch #{epoch} finished')

In [0]:
descriptor = Image2TextDescriptor()

In [0]:
path='data/Train_GCC-training.tsv'
batch_size = 128
num_workers = 16 # 14 ~ 5 sec; 4 ~ 11 sec; 6 ~ 9 sec; 16 ~ 4 sec
timeout = 2
gcc_dataset = GCCDataset(path=path,  
                         tokenizer=descriptor.tokenizer, 
                         max_length=descriptor.gpt2.config.n_positions - 10)
dataloader = get_dataloader(gcc_dataset, batch_size, descriptor.special_ids.pad, 
                            img_transform=transform, 
                            timeout=timeout, 
                            num_workers=num_workers)

In [0]:
train(descriptor, dataloader, lr=1e-4)

!!! model weights are loaded !!!


maxrss = 4372024
('cuda:0', torch.float32, (768,))		375
('cuda:0', torch.float32, (3072, 768))		60
('cuda:0', torch.float32, (3072,))		60
('cuda:0', torch.float32, (2304,))		60
('cuda:0', torch.float32, (768, 3072))		60
('cuda:0', torch.float32, (768, 2304))		60
('cuda:0', torch.float32, (768, 768))		60
('cuda:0', torch.float32, (512,))		20
('cuda:0', torch.float32, (256,))		20
('cuda:0', torch.float32, (128,))		20
('cuda:0', torch.float32, (64,))		20
('cuda:0', torch.int64, ())		20
('cuda:0', torch.float32, ())		14
('cuda:0', torch.float32, (2, 72, 12, 27, 64))		12
('cuda:0', torch.float32, (2, 71, 12, 25, 64))		12
('cuda:0', torch.uint8, (1, 1, 1024, 1024))		12
('cuda:0', torch.float32, (50260, 768))		5
('cuda:0', torch.float32, (1024, 768))		5
('cuda:0', torch.float32, (768, 1000))		5
('cuda:0', torch.float32, (64, 64, 3, 3))		4
('cuda:0', torch.float32, (512, 512, 3, 3))		3
('cuda:0', torch.float32, (256, 256, 3, 3))		3
('cuda:0', torch.float32, (128, 128, 3, 3))		3
('cpu', torch.f

/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:102: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


maxrss = 4416472
('cuda:0', torch.float32, (768,))		375
('cuda:0', torch.float32, (3072, 768))		60
('cuda:0', torch.float32, (3072,))		60
('cuda:0', torch.float32, (2304,))		60
('cuda:0', torch.float32, (768, 3072))		60
('cuda:0', torch.float32, (768, 2304))		60
('cuda:0', torch.float32, (768, 768))		60
('cuda:0', torch.float32, (512,))		20
('cuda:0', torch.float32, (256,))		20
('cuda:0', torch.float32, (128,))		20
('cuda:0', torch.float32, (64,))		20
('cuda:0', torch.int64, ())		20
('cuda:0', torch.float32, ())		14
('cuda:0', torch.float32, (2, 75, 12, 31, 64))		12
('cuda:0', torch.float32, (2, 72, 12, 27, 64))		12
('cuda:0', torch.uint8, (1, 1, 1024, 1024))		12
('cuda:0', torch.float32, (50260, 768))		5
('cuda:0', torch.float32, (1024, 768))		5
('cuda:0', torch.float32, (768, 1000))		5
('cuda:0', torch.float32, (64, 64, 3, 3))		4
('cuda:0', torch.float32, (512, 512, 3, 3))		3
('cuda:0', torch.float32, (256, 256, 3, 3))		3
('cuda:0', torch.float32, (128, 128, 3, 3))		3
('cpu', torch.f

KeyboardInterrupt: ignored

In [0]:
!cp weights/temp_weights drive/My\ Drive/image_captioning/weights/temp_weights

In [0]:
print(torch.cuda.memory_summary())